# Split folder where our data is present

In [19]:
import pandas as pd
import sys
import os
import json
import glob

In [2]:
df_train = pd.read_csv("train.csv")

In [3]:
df_train

,filename,width,height,class,xmin,ymin,xmax,ymax
0,1230939811.jpg,375.0,500.0,Adidas,129.0,326.0,257.0,423.0
1,1230939811.jpg,375.0,500.0,Adidas,137.0,336.0,243.0,395.0
2,144503924.jpg,280.0,210.0,Adidas,38.0,12.0,234.0,142.0
3,145074777.jpg,500.0,375.0,Adidas,22.0,52.0,471.0,326.0
4,2403699729.jpg,500.0,375.0,Adidas,27.0,5.0,342.0,244.0
...,...,...,...,...,...,...,...,...
6080,aug_3_232.jpg,259.0,194.0,tata,16.0,34.0,155.0,160.0
6081,aug_3_233.jpg,311.0,162.0,tata,125.0,41.0,309.0,118.0
6082,aug_3_234.jpg,260.0,194.0,tata,52.0,66.0,131.0,121.0
6083,aug_3_235.jpg,261.0,193.0,tata,3.0,55.0,149.0,155.0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6085 entries, 0 to 6084
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  6085 non-null   object 
 1   width     6085 non-null   float64
 2   height    6085 non-null   float64
 3   class     6085 non-null   object 
 4   xmin      6085 non-null   float64
 5   ymin      6085 non-null   float64
 6   xmax      6085 non-null   float64
 7   ymax      6085 non-null   float64
dtypes: float64(6), object(2)
memory usage: 380.4+ KB


In [6]:
categories = list(df_train['class'].unique())
categories

['Adidas',
 'Apple',
 'BMW',
 'Citroen',
 'Cocacola',
 'DHL',
 'Fedex',
 'Ferrari',
 'Ford',
 'Google',
 'HP',
 'Heineken',
 'Intel',
 'McDonalds',
 'Mini',
 'Nbc',
 'Nike',
 'Pepsi',
 'Porsche',
 'Puma',
 'RedBull',
 'Sprite',
 'Starbucks',
 'Texaco',
 'Unicef',
 'Vodafone',
 'Yahoo',
 'reliance',
 'tata']

In [7]:
categories_dict = {name:id_ for id_ ,name in enumerate(categories) }
categories_dict

{'Adidas': 0,
 'Apple': 1,
 'BMW': 2,
 'Citroen': 3,
 'Cocacola': 4,
 'DHL': 5,
 'Fedex': 6,
 'Ferrari': 7,
 'Ford': 8,
 'Google': 9,
 'HP': 10,
 'Heineken': 11,
 'Intel': 12,
 'McDonalds': 13,
 'Mini': 14,
 'Nbc': 15,
 'Nike': 16,
 'Pepsi': 17,
 'Porsche': 18,
 'Puma': 19,
 'RedBull': 20,
 'Sprite': 21,
 'Starbucks': 22,
 'Texaco': 23,
 'Unicef': 24,
 'Vodafone': 25,
 'Yahoo': 26,
 'reliance': 27,
 'tata': 28}

### we have anathore time verfication with images with csv 
####    we can directly do convert df to json but we don't do this 

In [10]:
json_dict_train = {"images": [], "type": "instances", "annotations": [], "categories": []}

In [11]:
%%time
path = 'split/train'
bnd_id = 1
image = []
grouped = df_train.groupby('filename')

for file in  glob.glob(f"{path}/*jpg"):
#     print(os.path.basename(file))
    image_name = os.path.basename(file)
    group_df = grouped.get_group(image_name)
    image_id = str(image_name.split(".")[0])
    width = int(group_df['width'].iloc[:1].values)
    height = int(group_df['height'].iloc[:1].values)
#     print(image_name)
    img_dict = {
        "file_name": image_name
        ,"height": height
        ,"width": width
        ,"id": image_id
    }
    json_dict_train["images"].append(img_dict)
    
    for index,_ in group_df.iterrows():
        categories_names = group_df.loc[index,"class"]
        categories_id = categories_dict[categories_names]
        xmin = int(group_df.loc[index,'xmin'])
        xmax = int(group_df.loc[index,'xmax'])
        ymax = int(group_df.loc[index,'ymax'])
        ymin = int(group_df.loc[index,'ymin'])
   
        
        assert xmax > xmin ,image_name # it is fine to it for verfication 
        assert ymax > ymin
        
        o_width = abs(xmax - xmin)
        o_height = abs(ymax - ymin)
        
        ann = {
                "area": o_width * o_height,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [xmin, ymin, o_width, o_height],
                "category_id": categories_id,
                "id": bnd_id,
                "ignore": 0,
                "segmentation": [],
            }
        json_dict_train["annotations"].append(ann)
        bnd_id = bnd_id + 1
        
        for cate, cid in categories_dict.items():
            cat = {"supercategory": "none", "id": cid, "name": cate}
            json_dict_train["categories"].append(cat)
        
    
    


Wall time: 10.3 s


In [12]:
json_dict_train

{'images': [{'file_name': '100.jpg', 'height': 237, 'width': 213, 'id': '100'},
  {'file_name': '101.jpg', 'height': 178, 'width': 282, 'id': '101'},
  {'file_name': '102.jpg', 'height': 167, 'width': 301, 'id': '102'},
  {'file_name': '103.jpg', 'height': 194, 'width': 260, 'id': '103'},
  {'file_name': '104.jpg', 'height': 168, 'width': 299, 'id': '104'},
  {'file_name': '105.jpg', 'height': 205, 'width': 246, 'id': '105'},
  {'file_name': '106.jpg', 'height': 168, 'width': 300, 'id': '106'},
  {'file_name': '107.jpg', 'height': 183, 'width': 275, 'id': '107'},
  {'file_name': '1075391489.jpg',
   'height': 66,
   'width': 180,
   'id': '1075391489'},
  {'file_name': '108.jpg', 'height': 167, 'width': 301, 'id': '108'},
  {'file_name': '108232382.jpg',
   'height': 332,
   'width': 500,
   'id': '108232382'},
  {'file_name': '108232417.jpg',
   'height': 332,
   'width': 500,
   'id': '108232417'},
  {'file_name': '109.jpg', 'height': 100, 'width': 100, 'id': '109'},
  {'file_name': 

In [14]:
with open("train.json",'w') as f:
    json_str = json.dumps(json_dict_train)
    f.write(json_str)

#### Test set

In [16]:
df_test = pd.read_csv("test.csv")

In [17]:
%%time
json_dict_test = {"images": [], "type": "instances", "annotations": [], "categories": []}
path = 'split/test'
bnd_id = 1
image = []
grouped = df_test.groupby('filename')

for file in  glob.glob(f"{path}/*jpg"):
#     print(os.path.basename(file))
    image_name = os.path.basename(file)
    group_df = grouped.get_group(image_name)
    image_id = str(image_name.split(".")[0])
    width = int(group_df['width'].iloc[:1].values)
    height = int(group_df['height'].iloc[:1].values)
#     print(image_name)
    img_dict = {
        "file_name": image_name
        ,"height": height
        ,"width": width
        ,"id": image_id
    }
    json_dict_test["images"].append(img_dict)
    
    for index,_ in group_df.iterrows():
        categories_names = group_df.loc[index,"class"]
        categories_id = categories_dict[categories_names]
        xmin = int(group_df.loc[index,'xmin'])
        xmax = int(group_df.loc[index,'xmax'])
        ymax = int(group_df.loc[index,'ymax'])
        ymin = int(group_df.loc[index,'ymin'])
   
        
        assert xmax > xmin ,image_name # it is fine to it for verfication 
        assert ymax > ymin
        
        o_width = abs(xmax - xmin)
        o_height = abs(ymax - ymin)
        
        ann = {
                "area": o_width * o_height,
                "iscrowd": 0,
                "image_id": image_id,
                "bbox": [xmin, ymin, o_width, o_height],
                "category_id": categories_id,
                "id": bnd_id,
                "ignore": 0,
                "segmentation": [],
            }
        json_dict_test["annotations"].append(ann)
        bnd_id = bnd_id + 1
        
        for cate, cid in categories_dict.items():
            cat = {"supercategory": "none", "id": cid, "name": cate}
            json_dict_test["categories"].append(cat)

Wall time: 1.06 s


In [18]:
with open("test.json",'w') as f:
    json_str = json.dumps(json_dict_test)
    f.write(json_str)